In [41]:
import theano
import theano.tensor as tt
import numpy as np

x = tt.tensor(name='x', dtype='floatX', broadcastable=(True, False))
y = tt.tensor(name='y', dtype='floatX', broadcastable=(False, True))
cache = {'x': x, 'y': y}
rep_cache = {i: tt.alloc(cache[i], y.shape[0], x.shape[1]) for i in cache.keys()}

# flat concatenation to use in Hessian
flat_params = tt.join(0, rep_cache['x'].flatten(), rep_cache['y'].flatten())

# reconstructed variables to use to compute obj
flat_x, flat_y = tt.split(flat_params, [x.shape[1] * y.shape[0]] * 2, n_splits=2)
new_x = flat_x.reshape([y.shape[0], x.shape[1]])
new_y = flat_y.reshape([y.shape[0], x.shape[1]])

obj = 3*new_x**2 + new_y ** 2 + new_x*new_y
new_vars = [new_x, new_y]

grad = theano.grad(obj.sum(), flat_params)
hess, u = theano.scan(lambda i, gp, p: theano.grad(gp[i], p),
                      sequences=tt.arange(flat_params.shape[0]),
                      non_sequences=[grad, flat_params])
ofunc = theano.function([cache['x'], cache['y']], obj, mode='FAST_RUN', on_unused_input='warn')
gfunc = theano.function([cache['x'], cache['y']], grad, mode='FAST_RUN', on_unused_input='warn')
hfunc = theano.function([cache['x'], cache['y']], hess, mode='FAST_RUN', on_unused_input='warn')
x1 = [[1., 2., 3., 4., 5.]]
x1 = [[1.,2.,3.]]
y1 = [[6.], [7.], [8.], [9.]]
y1 = [[4.], [5.]]
ores = ofunc(x1, y1)
gres = gfunc(x1, y1).reshape(2, 3, 2)
hres = hfunc(x1, y1).reshape(-1,6).sum(axis=-1).reshape(2,6,2).transpose(1,0,2).reshape(3,2,2,2)
print('obj: ', ores)
print('obj.shape: ', ores.shape)
print('grad: ', gres)
print('grad elements shape: ', [i.shape for i in gres])
print('hess shape: ', hres.shape)
print('hres: ', hres)

('obj: ', array([[ 23.,  36.,  55.],
       [ 33.,  47.,  67.]]))
('obj.shape: ', (2, 3))
('grad: ', array([[[ 10.,  16.],
        [ 22.,  11.],
        [ 17.,  23.]],

       [[  9.,  10.],
        [ 11.,  11.],
        [ 12.,  13.]]]))
('grad elements shape: ', [(3, 2), (3, 2)])
('hess shape: ', (3, 2, 2, 2))
('hres: ', array([[[[ 6.,  1.],
         [ 1.,  2.]],

        [[ 6.,  1.],
         [ 1.,  2.]]],


       [[[ 6.,  1.],
         [ 1.,  2.]],

        [[ 6.,  1.],
         [ 1.,  2.]]],


       [[[ 6.,  1.],
         [ 1.,  2.]],

        [[ 6.,  1.],
         [ 1.,  2.]]]]))
